# Read Data from the Cloud

```{article-info}
:author: Altay Sansal
:date: "{sub-ref}`today`"
:read-time: "{sub-ref}`wordcount-minutes` min read"
:class-container: sd-p-0 sd-outline-muted sd-rounded-3 sd-font-weight-light
```

In this tutorial, we will use a public SEG-Y file located in Amazon Web Services' (AWS)
Simple Storage Service (S3), also known as a cloud object store.

This dataset, the Parihaka 3D full angle stack (4.7 GB per volume including full angle
and near, mid, and far stacks), is provided by New Zealand Petroleum and Minerals (NZPM).
Available information and data acquisition details are accessible via the
[SEG Wiki][seg wiki], the [New Zealand GNS website], and the [NZPM data portal].

[seg wiki]: https://wiki.seg.org/wiki/Parihaka-3D
[nzpm data portal]: http://data.nzpam.govt.nz/GOLD/system/mainframe.asp
[new zealand gns website]: http://www.gns.cri.nz

```{important}
For plotting, the notebook requires [Matplotlib](https://matplotlib.org/) as a dependency.
Please install it before executing using `pip install matplotlib` or `conda install matplotlib`.
```

Let's start by importing some modules we will be using.

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import JSON
from numpy.random import default_rng

from segy import SegyFile
from segy.config import SegyFileSettings
from segy.schema import HeaderField
from segy.standards import rev1_segy

You can (but don't) download the SEG-Y directly clicking the [HTTP link] from the website.

[http link]: http://s3.amazonaws.com/open.source.geoscience/open_data/newzealand/Taranaiki_Basin/PARIHAKA-3D/Parihaka_PSTM_full_angle.sgy

This link is convenient as the `segy` library supports HTTP and we can directly use it
without downloading as well. Hovewer, For demonstration purposes, we'll use the 
corresponding S3 link (or called bucket and prefix):

`s3://open.source.geoscience/open_data/newzealand/Taranaiki_Basin/PARIHAKA-3D/Parihaka_PSTM_full_angle.sgy`

It's important to note that the file isn't downloaded but rather read on demand from the
S3 object store with the `segy` library. 

The `SegyFile` class uses information from the binary file header to construct a SEG-Y
descriptor, allowing it to read the file. The SEG-Y Revision is inferred from the binary
header by default, but can be manually set by providing a custom spec or adjusting settings.

Since this is a public bucket and an object, we need to tell `S3` that we want anonymous
access, which is done by configuring `storage_options` in settings.

In [ ]:
path = "s3://open.source.geoscience/open_data/newzealand/Taranaiki_Basin/PARIHAKA-3D/Parihaka_PSTM_full_angle.sgy"

# Alternatively via HTTP
# path = "http://s3.amazonaws.com/open.source.geoscience/open_data/newzealand/Taranaiki_Basin/PARIHAKA-3D/Parihaka_PSTM_full_angle.sgy"

settings = SegyFileSettings(storage_options={"anon": True})

sgy = SegyFile(path, settings=settings)

Let's investigate the JSON version of the SEG-Y spec for this file.

Some things to note:
1. The opening processed inferred Revision 1 from the binary header automatically.
2. It generated the spec using **default** SEG-Y Revision 1 headers.
3. We can check that some headers can be defined in the wrong byte locations.
4. There are too many headers to deal with in the default schema.

Note that we can build this JSON independently, load it into the descriptor
and open any SEG-Y with a schema.

In [ ]:
JSON(sgy.spec.model_dump(mode="json"))

Let's check the file size, number of traces, sample rate, etc. As expected, the file size
matches what was in the description. We also observe there are ~ 1 million traces.

In [ ]:
print(f"file size: {sgy.file_size / 1024**3:0.2f} GiB")
print(f"num traces: {sgy.num_traces:,}")
print(f"sample rate: {sgy.sample_interval}")
print(f"num samples: {sgy.samples_per_trace}")
print(f"sample labels: {sgy.sample_labels // 1000}")  # microsecond to millisecond

Using the `SegyFile` we can read SEG-Y components.

Here we read:
- Textual file header
- Binary file header
- 1,000 traces (headers + data) from somewhere in the middle of the file

In [ ]:
text_header = sgy.text_header
binary_header = sgy.binary_header

start = 500_000
stop = start + 1_000

traces = sgy.trace[start:stop]

trace_headers = traces.header
trace_data = traces.sample

This should take around one second or less, based on internet connection.

Let's print the textual header. There are a not many headers of interest. The
available headers appear to be in the Revision 1 byte locations.

In [ ]:
print(text_header)

We can look at headers (by default it is a Pandas `DataFrame`) in a nicely formatted table. 

We can also do typical Pandas analytics (like plots, statistics, etc.) but it won't be shown here.

In [ ]:
binary_header.to_dataframe()

In [ ]:
trace_headers.to_dataframe()

Let's plot the traces.

Note that they are all parsed from IBM floats to IEEE floats (decoded) in the background.

In [ ]:
plt.figure(figsize=(12, 8))
plot_kw = {"aspect": "auto", "cmap": "gray_r", "interpolation": "bilinear"}
plt.imshow(trace_data.T, vmin=-1000, vmax=1000, **plot_kw);

## With Custom Spec

We will create a new custom spec based on SEG-Y revision 1 with different binary
and trace header fields. This way, we will parse ONLY the parts we want, with
correct byte locations.

A user can define a completely custom SEG-Y spec from scratch as well, but for
convenience, we are customizing the Revision 1 schema with the parts that we
want to change.

Note that doing this will also modify the `segyStandard` field to `None` to
ensure we don't assume the file schema is standard after doing this.

From the binary file header, we will read:
- Number of samples
- Sample rate

From the trace headers, we will read:
- Inline
- Crossline
- CDP-X
- CDP-Y
- Coordinate scalar

Based on the text header lines:

```
C 2 HEADER BYTE LOCATIONS AND TYPES:                                            
C 3     3D INLINE : 189-192 (4-BYTE INT)    3D CROSSLINE: 193-196 (4-BYTE INT)  
C 4     ENSEMBLE X: 181-184 (4-BYTE INT)    ENSEMBLE Y  : 185-188 (4-BYTE INT)   
```

As we know by the SEG-Y Rev1 definition, the coordinate scalars are at byte 71.

In [ ]:
custom_spec = rev1_segy.customize(
    binary_header_fields=[
        HeaderField(name="sample_interval", byte=17, format="int16"),
        HeaderField(name="samples_per_trace", byte=21, format="int16"),
        HeaderField(name="ext_textual_headers", byte=305, format="int16"),
    ],
    trace_header_fields=[
        HeaderField(name="inline", byte=189, format="int32"),
        HeaderField(name="crossline", byte=193, format="int32"),
        HeaderField(name="cdp-x", byte=181, format="int32"),
        HeaderField(name="cdp-y", byte=185, format="int32"),
        HeaderField(name="coordinate_scalar", byte=71, format="int16"),
    ],
)

sgy = SegyFile(path, spec=custom_spec, settings=settings)

Now let's look at the JSON for the desciptor again. It is a lot more compact.

In [ ]:
JSON(sgy.spec.model_dump(mode="json"))

As mentioned earlier, the JSON can be laded into the spec from a file too.

```python
from segy.schema import SegySpec
import os

json_path = "..."

with open(json_path, mode="r") as fp:
    data = fp.read()
    spec = SegySpec.model_validate_json(data)
```

Let's do something a little more interesting now. Let's try to plot a time slice
by randomly sampling the file.

We will read 5,000 random traces. This should take about 15-20 seconds, based on
your internet connection speed.

In [ ]:
text_header = sgy.text_header
binary_header = sgy.binary_header

rng = default_rng()
indices = rng.integers(0, sgy.num_traces, size=5_000).tolist()

traces = sgy.trace[indices]

In [ ]:
binary_header.to_dataframe()

In [ ]:
trace_headers = traces.header.to_dataframe()

trace_headers["cdp-x"] /= trace_headers["coordinate_scalar"].abs()
trace_headers["cdp-y"] /= trace_headers["coordinate_scalar"].abs()

trace_headers

Now we can plot the time slice on actual coordinates from the headers and
even see a hint of the outline of the data! Since we significantly
down-sampled the data, the time slice is aliased and not very useful,
but this shows us the concept of making maps.

In [ ]:
plt.figure(figsize=(10, 8))

z_slice_index = 500

x, y, z = (
    trace_headers["cdp-x"],
    trace_headers["cdp-y"],
    traces.sample[:, z_slice_index],
)

scatter_kw = {"ec": [0.0, 0.0, 0.0, 0.5], "linewidth": 0.5}
color_kw = {"cmap": "gray_r", "vmin": -1000, "vmax": 1000}

plt.tripcolor(x, y, z, shading="gouraud", **color_kw)
plt.scatter(x, y, s=4, c=z, label="trace_locations", **scatter_kw, **color_kw)
plt.title("Water Bottom Map")
plt.colorbar(label="Amplitude")
plt.xlabel("X-Coordinate")
plt.ylabel("Y-Coordinate")
plt.legend();